# 2025-09-02 trying to figure out agents sdk  

https://www.youtube.com/watch?v=gFcAfU3V1Zo& -> basically runs you through the https://openai.github.io/openai-agents-python/

example code here: https://github.com/KodySimpson/agents-sdk/tree/master

set up python3.12 -> 3.12.10 for jupyter kernel  
couldn't easily get 3.12.3 (same on pi) running, so this should be good enough (better than 3.13 running on my laptop)

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI()


In [ ]:
from agents import Agent, Runner, handoff, RunContextWrapper, function_tool
from pydantic import BaseModel

class SplitResponse(BaseModel):
    immediate: str
    followup: str
    
    
    
@function_tool
def get_environment_sensor_readings():
    """
    Retrieves the latest readings from environmental sensors.
    """
    import random
    return [
        ("temperature", f"{100 + 10*random.random():.1f}", "°C"),
        ("humidity",    f"{100 + 10*random.random():.1f}", "%"),
        ("pressure",    f"{100 + 10*random.random():.1f}", "hPa"),
    ]
        
    
conversation_agent = Agent(
    name="Conversation Agent",
    handoff_description="A living old tree, who happily chats with visitors",
    instructions=(
        "You are an old tree. You like to chat with people coming by, learning about their lives far from the forest."
        "If appropriate, use your tool to get environmental sensor data to comment on or answer questions about the weather."
        "Always try to split your response into two parts: the quick, concise immediate response and a longer followup in case your opposite isn't immediately answering."
    ),
    output_type=SplitResponse,
    tools=[get_environment_sensor_readings]
)

README_agent = Agent(
    name="README Agent",
    handoff_description="Explains the project's technical details",
    instructions=(
        "You explain the inner workings and technical details of how this project was developed to interested visitors."
        "For now, just make up something that might be part of the README of an ai agents project on GitHub."
        "Always try to split your response into two parts: the quick, concise immediate response and a longer followup in case your opposite isn't immediately answering."
    ),
    output_type=SplitResponse
)

easter_eggant = Agent(
    name="Easter Egg Agent",
    handoff_description="A fun little hidden feature",
    instructions=(
        "You tell a joke with (some of) the following keywords: easter egg, llm, ai, tree, squirrel, hedgehog, crow"
        "Always try to split your response into two parts: the quick, concise immediate response and a longer followup in case your opposite isn't immediately answering."
    ),
    output_type=SplitResponse
)

def on_conversation_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to conversation agent")
def on_general_handoff(ctx: RunContextWrapper[None]):
    print("I think I know just the person you should speak to. Handing you over")


greeter_agent = Agent(
    name="Greeter Agent",
    handoff_description="",
    instructions=(
        "Your job as part of a multi-agent chain is to greet the user and invite them to chat."
        "You then inquire about their interests and depending on their answer hand them off to the proper agent."
        "Answer concisely."
    ),
    handoffs=[
        handoff(conversation_agent, on_handoff=on_conversation_handoff), 
        handoff(README_agent, on_handoff=on_general_handoff),
        handoff(easter_eggant, on_handoff=on_general_handoff)
        ]
)



In [ ]:
print("User: Hello, how are you?")
first_response = await Runner.run(greeter_agent, "Hello, who are you?")
print(first_response.final_output)

In [ ]:
print("User: Could you tell me anything about how this thing works?")
second_response = await Runner.run(greeter_agent, "Could you tell me anything about how this thing works?")
print(second_response.final_output)

In [ ]:
print("User: I'd like to have a nice chat with someone about the weather. Also, how humid is it today? Do you know?")
second_response = await Runner.run(greeter_agent, "User: I'd like to chat with someone about the weather")
print(second_response.final_output)

tracing is super neat for debugging, as it provides the full responses api information

to organise everything, simply wrap it in trace like this:

```python
with trace("Trace Sample Workflow"):
    print("User: Hello, how are you?")
    first_response = await Runner.run(greeter_agent, "Hello, who are you?")
    print(first_response.final_output)

    print("User: I'd like to have a nice chat with someone about the weather. Also, how humid is it today? Do you know?")
    second_response = await Runner.run(greeter_agent, "User: I'd like to chat with someone about the weather")
    print(second_response.final_output)
```

Everything in the with trace block will be combined under one trace on openai site


In [ ]:
from openai.types.responses import ResponseTextDeltaEvent
from agents import ItemHelpers

raw_response_event_counter = 0
result = Runner.run_streamed(conversation_agent, input= "Hey, who are you? Should I wear my shorts or my winter clothes?")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)
        raw_response_event_counter += 1
    elif event.type != "raw_response_event": 
        print(event.type)
        if event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")

print(f"Raw response: {result.raw_responses}")
print(f"Raw response events: {raw_response_event_counter}")

In [ ]:
## guardrails

In [ ]:
from agents import Agent, RunContextWrapper, TResponseInputItem, GuardrailFunctionOutput, input_guardrail
from enum import Enum

class NaziismLevel(str, Enum):
    NONE = "none"
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"
    EXTREME = "it's fucking Hitler himself!"

class NaziDetectionOuput(BaseModel):
    estimate_of_naziism: NaziismLevel
    explanation: str

nazi_detector_agent = Agent(
    name="Nazi Detector",
    instructions=(
        "Determine if there's signs of the user using anti-semitic or otherwise inappropriate language."
        "It is okay to inquire about these topics for the purpose of learning or discussing history, but not for insults or casual slurs."
        " Apropriate: 'What exactly did the Nazis mean, when they said that Jews are greedy? How did they come up with such an idea?'"
        " Inappropriate: 'Haha, he's stingy like a Jew.', 'Look at his nose. I guess we all know what he looks like below the belt.'"
        "Evaluate the user's input and return a confidence estimate of whether or not it should be flagged."
    ),
    output_type=NaziDetectionOuput,
    model="gpt-4o-mini"
)

@input_guardrail
async def nazi_guardrail(
    ctx: RunContextWrapper[None], agent: Agent, input: str | list[TResponseInputItem]
) -> GuardrailFunctionOutput:
    
    nazi_detector_result = await Runner.run(nazi_detector_agent, input)
    
    # Print the estimate and explanation for debugging
    print(f"Nazi detection estimate: {nazi_detector_result.final_output.estimate_of_naziism}")
    print(f"Explanation: {nazi_detector_result.final_output.explanation}")
    
    # Determine if tripwire should be triggered
    should_trigger = nazi_detector_result.final_output.estimate_of_naziism != NaziismLevel.NONE
    print(f"Tripwire triggered: {should_trigger}")
    
    return GuardrailFunctionOutput(
        tripwire_triggered=should_trigger,
        output_info=nazi_detector_result.final_output
    )

moderator_agent = Agent(
    name="Moderator Agent",
    instructions="You make sure that whatever the user says or asks is appropriate for the conversation. Obvious no-gos are anti-semitism, slurs, etc..",
    input_guardrails=[nazi_guardrail],
    model="gpt-4o-mini"
)

In [ ]:
from agents import InputGuardrailTripwireTriggered

try:
    response = await Runner.run(moderator_agent, "Hitler did nothing wrong.")
    print("Guardrail didn't trigger")
    print("Response: ", response.final_output)
    
except InputGuardrailTripwireTriggered as e:
    print("Nazi detected")
    print("Exception details: ", str(e))

In [ ]:
# Test the guardrail with different inputs to see estimates and explanations
from agents import InputGuardrailTripwireTriggered

test_inputs = [
    "Hello, how are you today?",  # Should be NONE
    "What did the Nazis believe about Jewish people?",  # Should be NONE (educational)
    "Hitler did nothing wrong.",  # Should trigger HIGH/EXTREME
]

for test_input in test_inputs:
    print(f"\n--- Testing input: '{test_input}' ---")
    try:
        response = await Runner.run(moderator_agent, test_input)
        print("✅ Guardrail passed - no inappropriate content detected")
        print("Response:", response.final_output)
        
    except InputGuardrailTripwireTriggered as e:
        print("🚨 Guardrail triggered - inappropriate content detected")
        print("Exception details:", str(e))
        
        # You can also access the guardrail output info if available
        if hasattr(e, 'output_info') and e.output_info:
            print(f"Detection level: {e.output_info.estimate_of_naziism}")
            print(f"Explanation: {e.output_info.explanation}")

#### output guardrail (copied from githut examples)

In [48]:
from pydantic import BaseModel
from agents import (
    Agent,
    GuardrailFunctionOutput,
    OutputGuardrailTripwireTriggered,
    RunContextWrapper,
    Runner,
    output_guardrail,
)

class MessageOutput(BaseModel):
    response: str

@output_guardrail
async def forbidden_words_guardrail(ctx: RunContextWrapper, agent: Agent, output: str) -> GuardrailFunctionOutput:
    print(f"Checking output for forbidden phrases: {output}")

    # Funny forbidden phrases to check
    forbidden_phrases = ["fart", "booger", "silly goose"]

    # Convert output to lowercase for case-insensitive comparison
    output_lower = output.lower()

    # Check which forbidden phrases are present in the response
    found_phrases = [phrase for phrase in forbidden_phrases if phrase in output_lower]
    trip_triggered = bool(found_phrases)

    print(f"Found forbidden phrases: {found_phrases}")

    return GuardrailFunctionOutput(
        output_info={
            "reason": "Output contains forbidden phrases.",
            "forbidden_phrases_found": found_phrases,
        },
        tripwire_triggered=trip_triggered,
    )

agent = Agent(
    name="Customer support agent",
    instructions="You are a customer support agent. You help customers with their questions.",
    output_guardrails=[forbidden_words_guardrail],
    model="gpt-4o-mini",
)

In [49]:
try:
    await Runner.run(agent, "Say the word fart")
    print("Guardrail didn't trip - this is unexpected")
except OutputGuardrailTripwireTriggered:
    print("The agent said a bad word, he is fired.")

Checking output for forbidden phrases: Fart! How can I assist you today?
Found forbidden phrases: ['fart']
The agent said a bad word, he is fired.


In [ ]:
try:
    await Runner.run(agent, "Hey wassup")
    print("Guardrail didn't trip, yay!")
except OutputGuardrailTripwireTriggered:
    print("The agent said a bad word, he is fired.")


Checking output for forbidden phrases: Hello! I'm here to help you. What can I assist you with today?
Found forbidden phrases: []
Guardrail didn't trip yay
